In [1]:
from CONFIG import *

In [2]:
import torch

In [3]:
import os

In [4]:
import torch.nn as nn

In [5]:
from torch.utils import model_zoo

In [6]:
from Our_VGG16 import vgg

In [7]:
from yoloLoss import yoloLoss

In [8]:
from torch.nn.init import xavier_uniform

In [9]:
import time

In [10]:
net = vgg(True)

In [11]:
net.batch_norm

True

In [12]:
if net.batch_norm:
    net.load_state_dict(model_zoo.load_url(vgg16_batch_norm_url))
else:
    net.load_state_dict(model_zoo.load_url(vgg16_url))


In [13]:
net.classifier = nn.Sequential(
                    nn.Linear(512*7*7,4096),
                    nn.ReLU(True),
                    nn.Dropout(),
                    nn.Linear(4096,1470))

In [14]:
for layer in net.classifier:
    print(layer)
    print("--------------------")
    if isinstance(layer,nn.Linear):
        xavier_uniform(layer.weight.data)

Linear(in_features=25088, out_features=4096, bias=True)
--------------------
ReLU(inplace)
--------------------
Dropout(p=0.5)
--------------------
Linear(in_features=4096, out_features=1470, bias=True)
--------------------


In [15]:
if torch.cuda.is_available():
    net.cuda()

In [16]:
net.train()

vgg(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True

In [17]:
optimizer = torch.optim.Adam(net.parameters(),lr = learning_rate,weight_decay=weight_decay)

In [18]:
optimizer2 = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

In [38]:
train_imgs_dir = 'VOC_Datasets/VOC_2012_training/VOC2012/JPEGImages/'
train_ann_dir = 'voc2012train.txt'
val_imgs_dir = 'VOC_Datasets/VOC_2007_testing/VOCdevkit/VOC2007/JPEGImages/'
val_ann_dir = 'voc2007test.txt'

In [39]:
from dataset import yoloDataset
import torchvision.transforms as transforms

In [40]:
img_names = os.listdir(train_imgs_dir)
train_dataset = yoloDataset(train_imgs_dir,train_ann_dir,train = True, transform = [transforms.ToTensor()])
val_dataset = yoloDataset(val_imgs_dir,val_ann_dir,train = False, transform = [transforms.ToTensor()])

data init
data init


In [41]:
from torch.utils.data import DataLoader

In [42]:
train_batches = DataLoader(train_dataset,batch_size = batch_size, shuffle = False, num_workers = 2)
val_batches = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 2)

In [43]:
len(train_dataset)

17125

In [44]:
len(val_dataset)

4952

In [45]:
print len(train_batches), len(val_batches)

536 155


In [46]:
from torch.autograd import Variable

In [47]:
yoloLossCriterion = yoloLoss(7,2,5,0.5)

In [48]:
train_loss_list = []
val_loss_list = []

In [49]:
def batch_process(mode,j):
    loss_list = []
    if mode == 'train':
        net.train()
        batches = train_batches
    else:
        net.eval()
        batches = val_batches
    for (images,targets) in batches:
        start = time.time()
        images,targets = Variable(images),Variable(targets)
        if torch.cuda.is_available():
            images = images.cuda()
            targets = targets.cuda()
        predictions = net(images)
        loss = yoloLossCriterion(predictions,targets)
    #         print(loss.data[0])
    
        loss_list.append(loss.data[0])
        if mode == 'train':
            if loss.data[0] < lowest_training_loss:
                torch.save(net.state_dict(),'training_weights.pth')
            optimizer2.zero_grad()
            loss.backward()
            optimizer2.step()
        else:
            if loss.data[0] < lowest_val_loss:
                torch.save(net.state_dict(),'validation_weights.pth')                
        j += 1
        end = time.time()
    #         if (j-1)%5 == 0:
        print "Epoch : ", i+1 , "/", num_epochs,"\n"
        print "Batch : ",j, "/",len(batches)," : Loss is :", loss.data[0]
        print "The time for this batch was :",(end - start)
    return loss_list

In [50]:
if 'validation_weights.pth' in os.listdir('./'):
    net.load_state_dict(torch.load('validation_weights.pth'))
lowest_training_loss = float("inf")
lowest_val_loss = float("inf")

i = 0
training_loss_list = []
val_loss_list = []
while i < num_epochs:
    j = 0
#     training_loss_list += batch_process('train',j)
    
    val_loss_list += batch_process('val',j)
    i += 1
        
    
            
        
        
    

Epoch :  1 / 100 

Batch :  1 / 155  : Loss is : 7.64876365662
The time for this batch was : 1.59772396088
Epoch :  1 / 100 

Batch :  2 / 155  : Loss is : 9.12448596954
The time for this batch was : 1.67223000526
Epoch :  1 / 100 

Batch :  3 / 155  : Loss is : 6.90010738373
The time for this batch was : 1.61173391342
Epoch :  1 / 100 

Batch :  4 / 155  : Loss is : 8.97832489014
The time for this batch was : 1.78799581528
Epoch :  1 / 100 

Batch :  5 / 155  : Loss is : 7.33425140381
The time for this batch was : 1.4803249836
Epoch :  1 / 100 

Batch :  6 / 155  : Loss is : 5.59162378311
The time for this batch was : 1.86843419075
Epoch :  1 / 100 

Batch :  7 / 155  : Loss is : 8.38409805298
The time for this batch was : 1.61028385162
Epoch :  1 / 100 

Batch :  8 / 155  : Loss is : 9.27209281921
The time for this batch was : 1.67858219147
Epoch :  1 / 100 

Batch :  9 / 155  : Loss is : 10.1722517014
The time for this batch was : 1.58207201958
Epoch :  1 / 100 

Batch :  10 / 155  

Epoch :  1 / 100 

Batch :  78 / 155  : Loss is : 9.80071640015
The time for this batch was : 1.64993810654
Epoch :  1 / 100 

Batch :  79 / 155  : Loss is : 9.62517642975
The time for this batch was : 1.53736996651
Epoch :  1 / 100 

Batch :  80 / 155  : Loss is : 6.86704111099
The time for this batch was : 1.55515599251
Epoch :  1 / 100 

Batch :  81 / 155  : Loss is : 8.06704044342
The time for this batch was : 1.73010921478
Epoch :  1 / 100 

Batch :  82 / 155  : Loss is : 8.4022693634
The time for this batch was : 1.58128499985
Epoch :  1 / 100 

Batch :  83 / 155  : Loss is : 6.86238384247
The time for this batch was : 1.65615105629
Epoch :  1 / 100 

Batch :  84 / 155  : Loss is : 6.57321977615
The time for this batch was : 1.69314002991
Epoch :  1 / 100 

Batch :  85 / 155  : Loss is : 5.3185210228
The time for this batch was : 1.72495007515
Epoch :  1 / 100 

Batch :  86 / 155  : Loss is : 9.58828735352
The time for this batch was : 1.58477711678
Epoch :  1 / 100 

Batch :  87

Epoch :  1 / 100 

Batch :  154 / 155  : Loss is : 8.18193817139
The time for this batch was : 1.83650898933
Epoch :  1 / 100 

Batch :  155 / 155  : Loss is : 8.69852161407
The time for this batch was : 1.83075690269
Epoch :  2 / 100 

Batch :  1 / 155  : Loss is : 7.64876365662
The time for this batch was : 1.51020002365
Epoch :  2 / 100 

Batch :  2 / 155  : Loss is : 9.12448596954
The time for this batch was : 1.63232302666
Epoch :  2 / 100 

Batch :  3 / 155  : Loss is : 6.90010738373
The time for this batch was : 1.60097098351
Epoch :  2 / 100 

Batch :  4 / 155  : Loss is : 8.97832489014
The time for this batch was : 1.86600613594
Epoch :  2 / 100 

Batch :  5 / 155  : Loss is : 7.33425140381
The time for this batch was : 1.67140007019
Epoch :  2 / 100 

Batch :  6 / 155  : Loss is : 5.59162378311
The time for this batch was : 1.73053312302
Epoch :  2 / 100 

Batch :  7 / 155  : Loss is : 8.38409805298
The time for this batch was : 1.62659788132
Epoch :  2 / 100 

Batch :  8 / 1

Epoch :  2 / 100 

Batch :  76 / 155  : Loss is : 9.33486557007
The time for this batch was : 1.70554208755
Epoch :  2 / 100 

Batch :  77 / 155  : Loss is : 7.18185806274
The time for this batch was : 1.6282851696
Epoch :  2 / 100 

Batch :  78 / 155  : Loss is : 9.80071640015
The time for this batch was : 1.73587107658
Epoch :  2 / 100 

Batch :  79 / 155  : Loss is : 9.62517642975
The time for this batch was : 1.78946900368
Epoch :  2 / 100 

Batch :  80 / 155  : Loss is : 6.86704111099
The time for this batch was : 1.94317293167
Epoch :  2 / 100 

Batch :  81 / 155  : Loss is : 8.06704044342
The time for this batch was : 1.52679514885
Epoch :  2 / 100 

Batch :  82 / 155  : Loss is : 8.4022693634
The time for this batch was : 1.79324293137
Epoch :  2 / 100 

Batch :  83 / 155  : Loss is : 6.86238384247
The time for this batch was : 1.55172491074
Epoch :  2 / 100 

Batch :  84 / 155  : Loss is : 6.57321977615
The time for this batch was : 1.70529913902
Epoch :  2 / 100 

Batch :  85

Epoch :  2 / 100 

Batch :  152 / 155  : Loss is : 11.6355714798
The time for this batch was : 2.25922584534
Epoch :  2 / 100 

Batch :  153 / 155  : Loss is : 7.97228479385
The time for this batch was : 1.72834014893
Epoch :  2 / 100 

Batch :  154 / 155  : Loss is : 8.18193817139
The time for this batch was : 1.65076613426
Epoch :  2 / 100 

Batch :  155 / 155  : Loss is : 8.69852161407
The time for this batch was : 1.74599003792
Epoch :  3 / 100 

Batch :  1 / 155  : Loss is : 7.64876365662
The time for this batch was : 1.75959515572
Epoch :  3 / 100 

Batch :  2 / 155  : Loss is : 9.12448596954
The time for this batch was : 1.64320802689
Epoch :  3 / 100 

Batch :  3 / 155  : Loss is : 6.90010738373
The time for this batch was : 1.81574106216


Process Process-13:
Traceback (most recent call last):
  File "/opt/conda/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Process Process-14:
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/opt/conda/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self._target(*self._args, **self._kwargs)
    self.run()
  File "/opt/conda/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
  File "/opt/conda/lib/python2.7/multiprocessing/process.py", line 114, in run
    r = index_queue.get()
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python2.7/multiprocessing/queues.py", line 376, in get
  File "/opt/conda/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    return recv()
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/conda/lib/python2.7/site-packag

KeyboardInterrupt: 

In [ ]:
loss_list

In [ ]:
type(loss_list)

In [ ]:
print("Epoch : ", i+1 , "/", num_epochs,"\n")